In [1]:
import duckdb

In [2]:
con = duckdb.connect("spark24.db")

In [3]:
script_ext_dir = "/opt/workspace/warc_yearly_pipeline/script_extraction_out/*.parquet"
maxmind_dir = "/opt/workspace/warc_yearly_pipeline/ipmaxmind_out/*.csv"

In [4]:
con.execute(f"CREATE TABLE IF NOT EXISTS script_extraction AS SELECT * FROM read_parquet('{script_ext_dir}')")

In [6]:
script_cnt = con.execute("SELECT COUNT(*) FROM script_extraction").fetchdf()
print(script_cnt)

   count_star()
0      22127510


In [17]:
con.execute(f"CREATE TABLE IF NOT EXISTS maxmind AS SELECT * FROM read_csv_auto('{maxmind_dir}', delim='\t', ignore_errors=true)")

In [18]:
maxmind_cnt = con.execute("SELECT COUNT(*) FROM maxmind").fetchdf()
print(maxmind_cnt)

   count_star()
0       9816625


In [20]:
con.execute(f"""
    CREATE TABLE IF NOT EXISTS cc24 AS
    SELECT *
    FROM script_extraction
    LEFT JOIN maxmind
    ON script_extraction.ip = maxmind.ip
    AND script_extraction.host = maxmind.host
    AND script_extraction.server = maxmind.server
    LIMIT 100000
""")

In [22]:
sample_join = con.execute("SELECT * FROM cc24").fetchdf()
sample_join.head()

,ip,host,server,emails,script_src_attrs,ip_1,host_1,server_1,postal_code,latitude,...,accuracy_radius,continent_code,continent_name,country_iso_code,country_name,subdivision_iso_code,subdivision_name,city_name,metro_code,time_zone
0,209.159.149.148,https://www.lenouveaureporter.com/etiquette/en...,Apache/2,,https://www.lenouveaureporter.com/wp-includes/...,209.159.149.148,https://www.lenouveaureporter.com/etiquette/en...,Apache/2,None,37.7510,...,1000.0,NA,North America,US,United States,None,None,None,NaN,America/Chicago
1,209.159.153.164,https://refundtalk.com/tag/transcript-processi...,Apache,,https://c0.wp.com/c/6.6.1/wp-includes/js/jquer...,209.159.153.164,https://refundtalk.com/tag/transcript-processi...,Apache,None,37.7510,...,1000.0,NA,North America,US,United States,None,None,None,NaN,America/Chicago
2,209.160.190.242,https://jp.infragistics.com/help/winforms/infr...,nginx/1.18.0,,/help/bundles/redirect-rules?v=c3g6-igtos4geyl...,209.160.190.242,https://jp.infragistics.com/help/winforms/infr...,nginx/1.18.0,08648,40.2152,...,100.0,NA,North America,US,United States,NJ,New Jersey,Lawrenceville,504.0,America/New_York
3,209.160.32.187,https://lemmy.sdf.org/post/9555466,nginx,,\'/static/1787a778/js/client.js\',209.160.32.187,https://lemmy.sdf.org/post/9555466,nginx,98160,47.6034,...,20.0,NA,North America,US,United States,WA,Washington,Seattle,819.0,America/Los_Angeles
4,209.17.116.10,http://loglink.com/search.asp?query=Western+Ca...,Microsoft-IIS/10.0,,,209.17.116.10,http://loglink.com/search.asp?query=Western+Ca...,Microsoft-IIS/10.0,None,37.7510,...,1000.0,NA,North America,US,United States,None,None,None,NaN,America/Chicago
